<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-LLM-ZeroToAll/02_prompt_engineering.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
! pip3 install -qU langchain-upstage

In [2]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Quick hello world
from langchain_upstage import ChatUpstage

llm = ChatUpstage()
llm.invoke("What's the best season to get to Korean?")

AIMessage(content="The best season to learn Korean is the season of your choice! The timing is ultimately up to you and depends on your personal preferences, schedule, and goals. Some people may find it helpful to set a specific goal, such as learning Korean in time for a trip to Korea or to watch a Korean drama or movie without subtitles.\n\nThat being said, if you're looking for general advice, it can be helpful to start learning Korean during a time when you have more free time and can dedicate yourself to consistent practice. This could be during a break from school or work, or during a time when you have fewer commitments.\n\nAdditionally, learning Korean can be a long-term commitment, so it's important to find a learning method that works for you and that you enjoy. This could be through online courses, language exchange programs, or even just watching Korean TV shows and movies.\n\nUltimately, the best season to get to Korean is when you're ready and motivated to start learning!

In [5]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
        ("ai", "I know of it. It's Paris!!"),
        ("human", "What about Korea?"),
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()
chain.invoke({})



'Oh, I believe the capital of Korea is Seoul.'

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria started with 23 apples. They used 20 to make lunch, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they added those to the 3 they had left. 3 + 6 = 9 apples.\n\nTherefore, the cafeteria has 9 apples.'

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

"The answer is 29.\n\nHere's the reasoning:\n\n1. The cafeteria started with 23 apples.\n2. They used 20 apples to make lunch, so they had 23 - 20 = 3 apples left.\n3. They bought 6 more apples, so they added those to the remaining apples.\n4. Therefore, they now have 3 + 6 = 9 apples."

![CoT](figures/cot.webp)

from https://arxiv.org/abs/2201.11903

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. Then they bought 6 more apples, so they had 3 + 6 = 9 apples. The answer is 9.'

![Zero-Shot COT](figures/zero-cot.webp)

From https://arxiv.org/abs/2205.11916

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'Step 1: The cafeteria started with 23 apples.\nStep 2: They used 20 apples to make lunch, so we subtract those from the initial amount: 23 - 20 = 3 apples left.\nStep 3: They then bought 6 more apples, so we add those to the remaining amount: 3 + 6 = 9 apples.\n\nA: The cafeteria has 9 apples now.'

## divide and conquer
![divideandconquer](figures/divideandconquer.png)

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. What is SOLAR 10.7B and what are its key features?\n2. How does depth up-scaling (DUS) work and what are its advantages over other LLM up-scaling methods?\n3. How does SOLAR 10.7B-Instruct compare to Mixtral-8x7B-Instruct in terms of instruction-following capabilities?'

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. SOLAR 10.7B\n2. depth up-scaling (DUS)\n3. instruction-following capabilities'

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "Depth up-scaling (DUS)":
    
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'What is the name of the large language model with 10.7 billion parameters introduced in the text, and what is its purpose?'

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [14]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

'Why did the chicken join the band?\n\nBecause it had the drumsticks!'

In [15]:
chain.invoke({"adjective": "funny", "content": "beef"})

'Why did the cow go to the party? To beef up the guest list!'

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "{keyword}":
    
    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the main method for scaling large language models (LLMs) introduced in the text, and how does it differ from other up-scaling methods?'

# Excercise 

Complete this code to create excellent questions from given documents using the following steps:
1. Include Few Shot examples, including CoT.
2. Generate keywords and topics first.
3. Iterate through each keyword and topic to generate questions.
4. Compare the generated questions with zero-shot generated questions.


# References
* https://platform.openai.com/docs/guides/prompt-engineering 
* https://docs.anthropic.com/claude/docs/intro-to-prompting 
* https://smith.langchain.com/hub 